In [ ]:
'''

2019. 11. 28.

by Yumi Kim CBBL at GNU
Screening file and Suggest Candidates which has good activity values

'''

import pandas as pd
import numpy as np
import pymysql.cursors

file_name_KCB_data = 'KCB_data_157694.csv'
data_df = pd.read_csv(file_name_KCB_data)

file_name_descriptor_list = 'qsar_descriptor_list.csv'
descriptor_list_df = pd.read_csv(file_name_descriptor_list)

file_name_all_qsar_df = 'qsar_model_list.csv'
all_qsar_df = pd.read_csv(file_name_all_qsar_df)

input_IC50_value = 0.5
screening_output_file_name = 'screening_result.csv'

def remove_specific_char(origine_srt, specific_char):
    return ''.join( c for c in origine_srt if  c not in specific_char )


def get_weight_list(weights):
    weights = remove_specific_char(weights, '[]')
    weights = weights.split(" ")

    weight_list = []
    
    for i in range(len(weights)):
        if(len(weights[i]) != 0):
            weight_list.append(float(weights[i]))
    
    return weight_list

# Get QSAR Weights and bias
bias = all_qsar_df['bias_vals'].values[0]
#bias = remove_specific_char(bias, '[]')
bias = float(bias)
weights = all_qsar_df['weights_vals'].values[0]

weight_list = get_weight_list(weights)

des_list = ['smile']
des_list_except_smile = []
for i in range(descriptor_list_df.shape[0]):
    des_list.append(descriptor_list_df['0'][i])
    des_list_except_smile.append(descriptor_list_df['0'][i])
    
optimized_df = data_df[des_list]

optimized_df["Pred_IC50"] = 0
for i in range(len(weight_list)):
    optimized_df["Pred_IC50"] += optimized_df[des_list_except_smile[i]]*weight_list[i]
optimized_df["Pred_IC50"]+= bias

optimized_df = optimized_df.sort_values(["Pred_IC50"], ascending=[True])
# Selection with Condition
low_value = optimized_df['Pred_IC50'] > 0
high_value = optimized_df['Pred_IC50'] < input_IC50_value
same_bias = optimized_df['Pred_IC50'] != bias
suggest_compounds_df = optimized_df[low_value & high_value & same_bias]
suggest_compounds_df.to_csv(screening_output_file_name, index=False)

In [4]:
suggest_compounds_df

,smile,AATS3m,ATSC2m,ATSC4v,C1SP3,VE3_Dt,Pred_IC50
4634,[F]C(O/C4=C/C=C(/OC[C@@]2([H])ON(C)[C@@](CN1N=...,77.326568,645.860725,-1331.578574,4,-22.229335,0.042854
4547,[F]C(OC=4C=CC(OC[C@@]1([H])ON(C)[C@](C1)(CN2C=...,78.293792,781.514713,-501.318218,5,-12.734697,0.105014
6469,O=C(CO/C1=C/C(=NN1C)C([F])([F])[F])N(/C2=C/C=C...,78.003740,746.924607,-444.810375,4,-7.924235,0.263157
4432,O=C(CC=1C=CC=CC=1OC[C@@]2([H])ON=C(C2)C3=C/C=C...,62.678599,121.943725,-698.673742,2,-8.145496,0.302673
1492,O=C2C(=C(NC=1C=CC(=CC=12)CCCCCC)[S](=O)C)C(=O)...,66.588609,201.593367,247.490130,2,-8.061243,0.340296
8364,[O-][N+](=O)C1=CC(=CC=C1)C(=O)NC(=O)C2=CC(C)=C...,80.816778,811.241902,-126.252151,4,-9.326891,0.383459
5397,O=C1[S]C(=NN1CC(=O)NC=2/C=C(/[Cl])C=CC=2)COC=3...,76.123970,585.493192,-219.920796,3,-10.560050,0.411889


In [2]:
all_qsar_df['bias_vals'].values[0]

459.18

In [6]:
all_qsar_df['weights_vals'].values[0]

'[[-7.4150e+00  2.2400e-01  1.2000e-02 -1.4506e+01 -1.9600e+00]]'